In [33]:
import http.client
import json
from datetime import datetime, timedelta
import csv
import pandas as pd
import requests

In [41]:
# Calculate current date and end date
current_date = datetime.now().strftime('%Y-%m-%d')
end_date = (datetime.now() + timedelta(days=14)).strftime('%Y-%m-%d')

# Construct URL with current and end dates
url = f"https://services.cehub.syngenta-ais.com/api/Forecast/ShortRangeForecastDaily?format=json&supplier=Meteoblue&startDate={current_date}&endDate={end_date}&measureLabel=TempAir_DailyAvg%20(C);TempAir_DailyMax%20(C);TempAir_DailyMin%20(C);Precip_DailySum%20(mm);WindDirection_DailyAvg%20(Deg);WindSpeed_DailyAvg%20(m/s);HumidityRel_DailyAvg%20(pct);WindDirection_DailyAvg;Soilmoisture_0to10cm_DailyAvg%20(vol%25);WindGust_DailyMax%20(m/s);Referenceevapotranspiration_DailySum%20(mm);TempSurface_DailyAvg%20(C);Soiltemperature_0to10cm_DailyAvg%20(C)&latitude=47&longitude=7"

payload = {}
headers = {
  'Accept': 'application/json, text/plain, */*',
  'ApiKey': 'b7b50ace-5063-4897-bfe5-436ec04b904b'
}

response = requests.request("GET", url, headers=headers, data=payload)
data = response.text
data_dict = json.loads(data)

# Convert to DataFrame
df = pd.DataFrame(data_dict)

# Rename columns
df.columns = ['latitude', 'longitude', 'date', 'measureLabel', 'dailyValue']

csv_file_path = 'climate_data.csv'
df.to_csv(csv_file_path, index=False)

     latitude  longitude                 date  \
0        47.0        7.0  2024/03/21 00:00:00   
1        47.0        7.0  2024/03/21 00:00:00   
2        47.0        7.0  2024/03/21 00:00:00   
3        47.0        7.0  2024/03/21 00:00:00   
4        47.0        7.0  2024/03/21 00:00:00   
..        ...        ...                  ...   
177      47.0        7.0  2024/04/03 00:00:00   
178      47.0        7.0  2024/04/03 00:00:00   
179      47.0        7.0  2024/04/03 00:00:00   
180      47.0        7.0  2024/04/03 00:00:00   
181      47.0        7.0  2024/04/03 00:00:00   

                                  measureLabel dailyValue  
0                         TempAir_DailyMin (C)          9  
1                         TempAir_DailyMax (C)       16.5  
2                         TempAir_DailyAvg (C)       12.1  
3                         Precip_DailySum (mm)        3.1  
4                   HumidityRel_DailyAvg (pct)         73  
..                                         ...     

In [ ]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
import pandas as pd

# Function to send email with attachment
def send_email(subject, body, to_email, attachment_path):
    from_email = "info@desky.digital"
    password = "Fabio<3Nobile"

    # Create message container
    msg = MIMEMultipart()
    msg['From'] = from_email
    msg['To'] = to_email
    msg['Subject'] = subject

    # Attach body to email
    msg.attach(MIMEText(body, 'plain'))

    # Attach CSV file
    filename = attachment_path.split('/')[-1]  # Extract filename from path
    attachment = open(attachment_path, "rb")
    part = MIMEBase('application', 'octet-stream')
    part.set_payload((attachment).read())
    encoders.encode_base64(part)
    part.add_header('Content-Disposition', "attachment; filename= %s" % filename)
    msg.attach(part)

    # Send the email
    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()
    server.login(from_email, password)
    text = msg.as_string()
    server.sendmail(from_email, to_email, text)
    server.quit()

# Email details
email_subject = 'Desky Farm - CSV Data'
email_body = 'Please find attached the CSV data containing several climate forecast of the next 2 weeks, we are thrilled to see what can you depict from them.'

# Email recipient
recipient_email = 'recipient@example.com'

# Send email with attachment
send_email(email_subject, email_body, recipient_email, csv_file_path)